In [42]:
ai_model = 'gemini-2.0-flash-001'
git_repo_name = "piotrhyzy/cloudfest2025"
git_pr_number = 6

In [43]:
import configuration as config

In [47]:
import vertexai
vertexai.init(project=config.VERTEX_PROJECT_ID, location=config.VERTEX_LOCATION)

In [48]:
from shared.gcp import Secret
secret = Secret.access_secret_version(
        config.GITHUB_SECRET_PROJECT_ID,
        config.GITHUB_SECRET_NAME,
    )
app_id = secret["app_id"]
installation_id = secret["installation_id"]
private_key = secret["private_key"]

In [49]:
from shared.git import get_client
git_client = get_client(app_id, installation_id, private_key, config.GITHUB_BASE_URL)

## Fetch PR Title and Description

In [ ]:
from shared.git import get_pr_details
pr_details = get_pr_details(git_client, git_repo_name, git_pr_number)

print(pr_details.model_dump_json(indent=2))

## Fetch PR Diff

In [ ]:
from shared.git import get_pr_diff
pr_diff = get_pr_diff(git_client, git_repo_name, git_pr_number)

print(pr_diff)


## Request LLM For Summary

In [52]:
from vertexai.generative_models import GenerativeModel, GenerationConfig


def summarize_pull_request(pr_title: str, pr_description: str, pr_diff: str) -> str:
    """
    Calls the Gemini model on Vertex AI to generate a concise summary of a pull request
    based on its title, description, and code diff.
    Returns the generated summary as a string.
    """

    # prepare configuration for the model
    generation_config = {
        "max_output_tokens": 8192, # limit the length of the response
        "temperature": 0.5, # lower temperature for more deterministic output
        "top_p": 1, # more deterministic output
        # "top_k": 30, # more deterministic output
    }
    # Load the Gemini model
    model = GenerativeModel(ai_model)

    # Build the prompt in English (basic instructions, no explicit "role")
    prompt_text = f"""
        Below is a pull request:

        Title:
        {pr_title}

        Description:
        {pr_description}

        Code Diff:
        {pr_diff}

        Task:
        1. Read the pull request details and code diff.
        2. Provide a concise summary of what this PR changes and why it matters with Highlights.
        3. Keep it short, focusing on the main modifications and purpose.
        4. Provide Changeog using expandalbe "Changelog" section for more details.

        Summary of Changes:
        """

    # Call the model with the prompt
    response = model.generate_content(
        prompt_text,
        generation_config=generation_config,
    )
    response_text = response.text.strip()

    # Strip the code fences if present
    if response_text.startswith('```json'):
        response_text = response_text[7:]
    if response_text.endswith('```'):
        response_text = response_text[:-3]
    response_text = response_text.strip()


    # Return the model's output as a string
    return response.text


In [ ]:
pr_summary = summarize_pull_request(pr_details.title, pr_details.description, pr_diff)
print(pr_summary)

In [ ]:
from shared.git import post_comment
post_comment(git_client, git_repo_name, git_pr_number, pr_summary)

## Chunk GitHub Diff

In [ ]:
from shared.diff import process_diff
unified_diff = process_diff(pr_diff)
print(unified_diff.model_dump_json(indent=4))

## Convert thr strctured diff to a review File object

In [56]:
from shared.review import convert_diff_to_review_files
review_files = convert_diff_to_review_files(unified_diff, pr_details)

In [ ]:
for _review_file in review_files:
    print(_review_file.model_dump_json(indent=2))

## Fetch File Snapschot

In [ ]:
from shared.git import download_file_snapschot_from_pr

for _review_file in review_files:
    _review_file.snapshot = download_file_snapschot_from_pr(git_client, git_repo_name, git_pr_number, _review_file.path)
    print(_review_file.model_dump_json(indent=2))

## Request LLM For File Review

In [59]:
from vertexai.generative_models import GenerativeModel

def review_single_file_changes_gemini(
    file_path: str,
    original_file_content: str,
    diff: str,
    language: str = 'python'
) -> str:
    """Reviews changes in a single file using the Gemini Pro model via Vertex AI SDK.

    Args:
        file_path: The path to the file.
        original_file_content: The original content of the file.
        diff: The diff of the changes.
        language: The programming language of the code.

    Returns:
        The review of the changes as a string.

    Raises:
        ValueError: If there are issues with the input parameters or the API response.
    """

    if not file_path:
        raise ValueError("file_path is required")
    if not original_file_content:
        raise ValueError("original_file_content is required")
    if not diff:
        raise ValueError("diff is required")
    if not language:
        raise ValueError("language is required")

    prompt_template = """
    You are an expert software engineer tasked with reviewing a proposed code change.
    Your role is to identify potential issues, suggest improvements,and assess the overall quality of the changes.

    The context is a single file. You are provided with the file's original content, the changes made (in diff format), and the file's path.

    **Instructions:**

    1.  **Understand the Changes:** Carefully analyze the provided diff to understand what modifications have been made to the file.
    2.  **Consider the Context:** Use the original file content to gain a better understanding of the changes in their existing codebase.
    3.  **Focus on Code Quality:** Evaluate the changes with respect to:
        *   **Correctness:** Do the changes function as intended? Are there any bugs or logical flaws?
        *   **Readability:** Is the code easy to understand? Are variable names meaningful? Is the code well-formatted?
        *   **Maintainability:** Are the changes maintainable? Are they well-structured?
        *   **Efficiency:** Are there any performance bottlenecks or potential inefficiencies introduced?
        *   **Security:** Are there any security vulnerabilities introduced?
        *   **Best Practices:** Do the changes adhere to best practices for the {language} programming language?
        * **Completeness** Are the changes complete? Is there missing logic?
    4.  **Provide Specific Feedback:**
        *   If you identify any issues, be specific about their location (e.g., "Line 25: The variable name 'x' is not descriptive.").
        *   Provide concrete suggestions for improvement.
        *   Explain the rationale behind your feedback.
    5.  **Be Concise:** Keep your review focused and avoid unnecessary filler.
    6. Be polite and respectful.

    **Input:**

    *   **File Path:** {file_path}
    *   **Language:** {language}
    *   **Original File Content:**
    ```
    {original_file_content}
    ```
    *   **Diff:**
    ```
    {diff}
    ```

    **Output:**

    Provide a comprehensive code review of the changes, in markdown, following the instructions above.
    """

    prompt = prompt_template.format(
        file_path=file_path,
        original_file_content=original_file_content,
        diff=diff,
        language=language,
    )

    # Configure the Gemini Pro model
    generation_config = {
        "candidate_count": 1,
        "max_output_tokens": 2048,
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 40,
    }
    # Load the Gemini model
    model = GenerativeModel(ai_model)

    # Call the model with the prompt
    response = model.generate_content(
        prompt,
        generation_config=generation_config,
    )
    response_text = response.text.strip()

    # Return the model's output as a string
    return response.text



In [60]:
file_reviews = []
for _review_file in review_files:
    file_reviews.append(
        review_single_file_changes_gemini(original_file_content=_review_file.snapshot, diff=_review_file.diff, file_path=_review_file.path)
    )

In [ ]:
print([file_reviews])

## Merge Reviews into a single review summary

In [62]:
def review_merged_files_changes(
    reviews: list[str],
) -> str:
    """Reviews changes in multiple files using the Gemini Pro model via Vertex AI SDK.

    Args:
        reviews: A list of code review strings (output from review_single_file_changes_gemini).

    Returns:
        A merged review summary as a string.

    Raises:
        ValueError: If there are issues with the input parameters or the API response.
    """

    if not reviews:
        raise ValueError("reviews list is empty")

    prompt_template = """
    You are an expert software engineer tasked with reviewing a set of code changes across multiple files.
    You have received individual code reviews for each file modified in this pull request. Your role is to:

    1.  **Synthesize Information:** Combine the feedback from all the individual file reviews into a coherent overall assessment of the pull request.
    2.  **Identify Major Changes:** Highlight the key changes across all the files.
    3.  **Assess Overall Quality:** Provide an overall assessment of the quality of the changes in the pull request, including:
        *   **Correctness:** Are the changes likely to work as intended, across all files?
        *   **Readability:** Is the code easy to read and understand across all the files?
        *   **Maintainability:** Are the changes maintainable across all the files?
        *   **Efficiency:** Are there any performance issues across all the files?
        *   **Security:** Are there any security issues across all the files?
        *   **Completeness:** Is there any missing logic across all the files?
    4.  **Provide Merge Assessment:** Based on your analysis, provide a recommendation on whether these changes are ready to be merged, or if further work is needed.
    5. **Format Output for Github:** return a text in a nice format understandable by github (using markdown, bold, italic, emoji, ...).
    6. Be polite and respectful.

    **Input:**

    *   **Individual File Reviews:**
    {reviews_text}

    **Output:**

    Provide a comprehensive summary review of the changes across all files, in markdown, following the instructions above.
    Include sections for:
    *   **Major Changes**
    *   **Issues**
    *   **Overall Quality Assessment**
    *   **Merge Assessment**

    Provide only output no other comments or intorductions start with:

    # Review of Changes:
    """

    reviews_text = "\n".join(
        [f"```\n{review}\n```" for review in reviews]
    )  # Format each review nicely

    prompt = prompt_template.format(reviews_text=reviews_text)

    # Configure the Gemini Pro model
    generation_config = {
        "candidate_count": 1,
        "max_output_tokens": 4096,  # Increased max output tokens
        "temperature": 0.2,
        "top_p": 1,
        "top_k": 40,
    }
    # Load the Gemini model
    model = GenerativeModel(ai_model)

    # Call the model with the prompt
    response = model.generate_content(
        prompt,
        generation_config=generation_config,
    )
    response_text = response.text.strip()

    # Return the model's output as a string
    return response.text

In [63]:
file_reviews_text =review_merged_files_changes(file_reviews)

In [ ]:
print(file_reviews_text)

## Publish Summary to GitHub

In [ ]:
from shared.git import post_comment
post_comment(git_client, git_repo_name, git_pr_number, file_reviews_text)